In [1]:
!nvidia-smi

Sat Sep 14 23:12:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:1E.0 Off |                    0 |
| N/A   45C    P0              34W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os

In [3]:
# Space I have 
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         100G   43G   58G  43% /
tmpfs            64M     0   64M   0% /dev
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  100G   43G   58G  43% /run
tmpfs            14G     0   14G   0% /dev/shm
/dev/nvme2n1    2.0G  1.9G     0 100% /home/jovyan
tmpfs            14G  120K   14G   1% /home/jovyan/.saturn
tmpfs            14G   12K   14G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.7G   12K  7.7G   1% /proc/driver/nvidia
tmpfs           7.7G  3.0M  7.7G   1% /run/nvidia-persistenced/socket
tmpfs           7.7G     0  7.7G   0% /proc/acpi
tmpfs           7.7G     0  7.7G   0% /sys/firmware


In [1]:
# Checking all the environment variables available
#os.environ

In [5]:
# As hugging face stores the file inside the cache folder inside HOME directory which in our case doesn't have mmuch storage, so changing the default storage of Hugging Face
# HF_HOME is the variable which contains the location to store the Hugging Face data
# MAKE SURE TO RESTART THE KERNEL AFTER THIS
os.environ['HF_HOME'] = '/run/cache/'

In [6]:
os.environ['HF_HOME']

'/run/cache/'

# Copied from Hugging Face <a href="https://huggingface.co/microsoft/Phi-3-mini-128k-instruct"> Webage of the model</a> 

## <span style="color:red"> Requirements as mentioned in the documentation

In [7]:
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers  # This is Hugging face Transformers
!pip install flash_attn==2.5.8
!pip install torch==2.3.1
!pip install accelerate==0.31.0

Found existing installation: transformers 4.45.0.dev0
Uninstalling transformers-4.45.0.dev0:
  Successfully uninstalled transformers-4.45.0.dev0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-vy40zmi6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-vy40zmi6
  Resolved https://github.com/huggingface/transformers to commit 8bd2b1e8c23234cd607ca8d63f53c1edfea27462
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9732484 sha256=c2c07201797458bc2bfe3a74d811e0dc0eda898deee353bb182237c3f30eb522
  Stored in directory: /tmp/pip-ephem-wheel-cache-7fuux0ac/wheels/14/a0/7b/8f6b25ba4110aa215fcb8d6aedd6cd4f9b9b6619190999ac2b
Successfully built transformers


In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [9]:
torch.random.manual_seed(0) 

In [10]:
model = AutoModelForCausalLM.from_pretrained( 
    "microsoft/Phi-3-mini-128k-instruct",  
    device_map="cuda",  
    torch_dtype="auto",  
    trust_remote_code=True,  
) 

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct") 

`flash-attention` package not found, consider installing for better performance: /opt/saturncloud/envs/saturn/lib/python3.9/site-packages/flash_attn_2_cuda.cpython-39-x86_64-linux-gnu.so: undefined symbol: _ZN2at4_ops5zeros4callEN3c108ArrayRefINS2_6SymIntEEENS2_8optionalINS2_10ScalarTypeEEENS6_INS2_6LayoutEEENS6_INS2_6DeviceEEENS6_IbEE.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
pipe = pipeline( 
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
) 

In [12]:
messages = [ 
    {"role": "user", "content": "How many days are there in a leap year and why?"}, ] 


generation_args = { 
    "max_new_tokens": 500, 
    "return_full_text": False, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

output = pipe(messages, **generation_args) 
print(output[0]['generated_text'])


/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/transformers/generation/utils.py:1346: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:599: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


 A leap year has 366 days instead of the usual 365 days. This extra day is added to February, making it 29 days long instead of the usual 28. The reason for this adjustment is to keep our calendar in alignment with the Earth's revolutions around the Sun. It takes the Earth approximately 365.24 days to complete one orbit around the Sun. Without the addition of a leap day every four years, our calendar would slowly drift away from the actual seasons. The Earth's orbit around the Sun takes approximately 365.24 days. To keep our calendar in sync with the Earth's revolutions, an extra day is added to the calendar every four years. This extra day is added to February, making it 29 days long instead of the usual 28. This adjustment ensures that our calendar remains aligned with the Earth's seasons. A leap year has 366 days, with the extra day added to February. This adjustment is necessary to keep our calendar in sync with the Earth's revolutions around the Sun. The Earth takes approximately 

# <span style="color:blue">Combining everything together to form a RAG

In [13]:
# Getting the Similar Doc Search python file created in Part_01_Basics
!rm -f similarDocSearch.py
!wget https://raw.githubusercontent.com/vaibhavcodes/LLM_RAG/main/Part_01_Basics/similarDocSearch.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2024-09-14 23:13:23--  https://raw.githubusercontent.com/vaibhavcodes/LLM_RAG/main/Part_01_Basics/similarDocSearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3320 (3.2K) [text/plain]
Saving to: ‘similarDocSearch.py’

similarDocSearch.py 100%[===================>]   3.24K  --.-KB/s    in 0s      

2024-09-14 23:13:23 (51.3 MB/s) - ‘similarDocSearch.py’ saved [3320/3320]



In [14]:
from similarDocSearch import *
import json
import requests

In [15]:
# Documents fetching from the github
docs_url = "https://github.com/vaibhavcodes/LLM_RAG/blob/main/Part_01_Basics/data/documents.json?raw=1"
docs_response = requests.get(docs_url)
docs_raw = docs_response.json()
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course'] 
        documents.append(doc)

In [16]:
# Initializing the class similarDocSearch
obj = similarDocSearch(text_fields = ["question", "text", "section"])
# Fitting
obj.fit(documents)

In [17]:
# Function that would search the most similar documents to our query and will return it
def search(query):
    boost = {'question': 3, 
         'section': 0.5} # Used to give a field more or less importance 
    
    filter_course = "data-engineering-zoomcamp"
    num_results = 5
    
    results = obj.search(query = query, boosts=boost, filter_course=filter_course, num_results=num_results)
    return results

In [18]:
# Function that would build up the prompt
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contains the answer, output None
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()

    # Context creation to be passed into the prompt
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\n question: {doc['question']}\n answer: {doc['text']}\n \n"
    
    # Prompt formatting
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return(prompt)

In [22]:
# Function that would use LLM
def llm(prompt):
    messages = [ 
    {"role": "user", "content": prompt} ] 


    generation_args = { 
        "max_new_tokens": 100, 
        "return_full_text": False, 
        "temperature": 0.0, 
        "do_sample": False, 
    } 

    output = pipe(messages, **generation_args) 
    return(output[0]['generated_text'])

In [23]:
# Function that would make a whole RAG
def RAG(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [24]:
q = "I just discovered the course. Can i still join it??"
print(RAG(q))

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:599: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 You can still join the course even if you discover it after the start date. You are eligible to submit the homeworks, but remember to meet the deadlines for the final projects. I'm sorry, but the provided context does not contain information about joining the course after the start date. The answer given is based on the FAQ database, which does not have the specific information requested in the question. I'm sorry, but the provided context does not contain
